This example uses a python library for working with an IAE instance.

Find the library documentation here: http://ibm-analytics-engine-python.readthedocs.io/en/latest/

In [ ]:
! pip install --quiet --upgrade git+https://github.com/snowch/ibm-analytics-engine-python@master

In [ ]:
from ibm_analytics_engine import CloudFoundryAPI, CloudFoundryAPI
from ibm_analytics_engine import IAE, IAEServicePlanGuid

In [ ]:
# Download your IBM Cloud API Key and save it in a secure location

cf = CloudFoundryAPI(api_key_filename='/Users/snowch/.apiKeys/csnow.json')

In [ ]:
# Provide your organizaton name and space name:

SPACE_GUID = cf.space_guid(org_name='my_org_name', space_name='my_space_name')
print(SPACE_GUID)

In [ ]:
# If you couldn't find your space guid, uncomment and run the following cell to print
# out all your orgs and spaces

# cf.print_orgs_and_spaces()

In [ ]:
iae = IAE(cf_client=cf)

In [ ]:
# List the clusters in the space

iae.clusters(space_guid=SPACE_GUID)

In [ ]:
cluster_guid = iae.create_cluster(service_instance_name='MY_HADOOP_CLUSTER', 
                   service_plan_guid=IAEServicePlanGuid.LITE, 
                   cluster_creation_parameters={
                        "hardware_config": "default",
                        "num_compute_nodes": 1,
                        "software_package": "ae-1.0-spark",
                    }, 
                   space_guid=SPACE_GUID)

# Alternative options for service_plan_guid:
    
# IAEServicePlanGuid.STD_HOURLY
# IAEServicePlanGuid.STD_MONTHLY

In [ ]:
import time
while True:
    status = iae.status(cluster_instance_guid=cluster_guid)
    print(status)
    if status == 'succeeded' or status == 'failed': break
    time.sleep(60)


In [ ]:
# Only run this cell after the previous cell has finished with the status 'succeeded',
# otherwise you will receive an error trying to get or create the credentials.

import json

# get the credentials data for the cluster in vcap json format
vcap = iae.get_or_create_credentials(cluster_instance_guid=cluster_guid)

# print the credentials out
vcap_formatted = json.dumps(vcap, indent=4, separators=(',', ': '))
print(vcap_formatted)

# save the credentials to a file
with open('./vcap.json', 'w') as vcap_file:
    vcap_file.write(vcap_formatted)

In [ ]:
# Grab the ambari console url

print(vcap['cluster']['service_endpoints']['ambari_console'])

In [ ]:
# Delete the cluster.  Recursive=True will delete service bindings, service keys,
# and routes associated with the service instance.

iae.delete_cluster(cluster_guid, recursive=True)